# Workshop_10

Esempio 1 

<img src="https://s29.postimg.org/8tvhpfion/w101.png">

Esempio 2

<img src="https://s28.postimg.org/ka0zebk7x/w102.png">

Esempio 3

<img src="https://s24.postimg.org/6ovi78pid/w103.png">

Esempio 4

<img src="https://s24.postimg.org/kt737q5h1/w105.png">

Esempio interno di un appartamento con porte e finestre

<img src="https://s27.postimg.org/cffwdk6lv/w104.png">


In [1]:
from pyplasm import *
from math import *
from __future__ import division
from pyplasm import *
#"""Vecchio workshop che crea le scale"""

#"creo un gradino dove width è la larghezza, riser è l'alzata e tread è la pedata"
def step(width, riser, tread) :
    step = MKPOL([[[0,0],[0,riser*2],[tread,riser*2],[tread,riser]],[[1,2,3,4]],1])
    step = PROD([QUOTE([width]),step])
    return step

#VIEW(step(1,0.2,0.25))
#"creo la scala come una struttura di gradini"
def stairs(width,riser,tread,n) :
    structureElements = [CUBOID([width,tread,riser]),T([2])([tread]),step(width,riser,tread)]
    for i in range(2,int(n)):
        structureElements.append(T([2,3])([tread,riser]))
        structureElements.append(step(width,riser,tread))
    s = STRUCT(structureElements)
    return s

#"metodo che crea 2 piattaforme, una per il pavimento una per il pianerottolo"
def platform(dx,dy,dz):
    s = stairs(dx,dy,dz,10)
    platform = CUBOID([2,1,0.2])
    upperPlatform = STRUCT([s,T([2,3])([2.5,1.8]),platform])
    structure = STRUCT([upperPlatform,s])
    floor=CUBOID([2,1,0.2])
    finalStructure = STRUCT([structure,T(2)(-1),floor])
    return finalStructure
#VIEW(platform())

#"metodo che crea un corrimano; usato con dati preimpostati"
def handrail(dx,dy,dz):
    handrail = MKPOL([[[0,1],[0.3,1],[2,2.8],[2.5,2.8]],[[1,2],[2,3],[3,4]],1])
    handrail = PROD([QUOTE([0.2]),handrail])
    p = platform(dx,dy,dz)
    handrail = STRUCT([handrail,T(1)(0),p])
    rod = MKPOL([[[0,0.2],[0,1]],[[1,2]],1])
    rod = PROD([QUOTE([0.2]),rod])
    handrail = STRUCT([handrail,T(1)(0),rod])
    rod1 = MKPOL([[[2.5,2.8],[2.5,2]],[[1,2]],1])
    rod1 = PROD([QUOTE([0.2]),rod1])
    handrail = STRUCT([handrail,T(1)(0),rod1])
    return handrail

def ggpl_modern_staircase(dx,dy,dz):
    return stairs(dx,dy,dz,10)

Evaluating fenvs.py..
...fenvs.py imported in 0.0226617134101 seconds


In [2]:
#"""Vecchio workshop relativo alla creazione di porte e finestre"""

def ggpl(X,Y,Z,occupancy):
    def dimension(dx,dy,dz):
        def color(cod):
            "preparo il ritorno e imposto a 0 l'indice e la somma dell'insieme X"
            finalStructure = CUBOID([0,0,0])
            final = CUBOID([0,0,0])
            indexX= 0
            sumX = 0
            
            for p in occupancy:
                cubo = CUBOID([0,0,0])
                "imposto a 0 l'indice e la somma dell'insieme  Y"
                indexZ =0
                sumZ = 0
                
                for riga in p:
                    "imposto a 0 l'indice e la somma dell'insieme  Z e creo un array temporaneo di supporto"
                    temp=[]
                    indexY=0
                    boolean = 1
                    
                    for val in riga:
                        "verifico che tipo di materiale è"
                        if val == cod:
                            temp.append(Y[indexY])
                            boolean = 0
                        else:
                            temp.append(-Y[indexY])
                        indexY+=1
                        
                    if not boolean:
                        "creo prima la quota della struttura "
                        qy = QUOTE(temp)
                        qz = QUOTE([Z[indexZ]])
                        pxy = PROD([QUOTE([X[indexX]]),qy])
                        pyz= PROD([pxy,qz])
                        finalStructure=STRUCT([finalStructure,T([3])([sumZ]),pyz])
                    "aumento l'indice"
                    sumZ+= Z[indexZ]
                    indexZ+=1
                "creo la struttura finale "
                final = STRUCT([final,T([1])([sumX]),finalStructure])
                sumX+=X[indexX]
                indexX+=1
                
            return finalStructure
        "colori dei materiali"
        glass = [210/255,1,1]
        wood = [80.0/255,41/255,0/255]
        woodS= COLOR(wood)(color(1))
        glasS= COLOR(glass)(color(2))
        
        return STRUCT([glasS,woodS])
    return dimension


Creo delle funzioni di supporto:
makePoints: serve a prendere un array di punti per poi scalarli
walls: funzione che dati dei punti restituisce un hpc che rappresenta i muri dell'appartamento
findMaxMin: serve a trovare i 4 punti del rettangolo che rappresenta l'appartamento
floor: restituisce il pavimento dell'appartamento
pointsReader: prende in input un file.txt di linee e restituisce un array di punti
doors,windows: metodi simili che restituiscono un OFFSET, rispettivamente di porte e finestre, date con dei file .txt in input

In [9]:
#"""restituisce un array di punti scalati e interi a partire da un array di punti passati in input"""
def makePoints(array):
    scale = 50
    l = len(array)
    points = []
    j = 0
    while(j<l):
        point = (int(array[j]/scale),int(array[j+1])/scale,0)
        points.append(point)
        j = j+2
    return points

#"""crea un muro a partire da un array di punti passati in input"""
def walls(array):
    l = len(array)
    wall = CUBOID([0,0,0])
    j = 1
    temp = []
    while(j<l):
        temp.append([j,j+1])
        j+=2
    wall =MKPOL([array,temp,1])
    wall = OFFSET([.1,.1,3])(wall)
    return wall

def findMaxMin(array):
    l = len(array)
    minx, miny, z = array[0]
    maxx, maxy, z = array[0]
    for i in range(0,l):
        x,y,z= array[i]
        if(x<minx):
            minx = x
        if(x>maxx):
            maxx= x
        if(y<miny):
            miny = y
        if(y>maxy):
            maxy= y
    temp = [(minx,miny,0),(maxx,miny,0),(minx,maxy,0),(maxx,maxy,0)]
    return temp

#"""crea un pavimento a partire da un array di punti passati in input"""
def floor(array):
    l = len(array)
    floor = CUBOID([0,0,0])
    minx, miny, z = array[0]
    maxx, maxy, z = array[0]
    for i in range(0,l):
        x,y,z= array[i]
        if(x<minx):
            minx = x
        if(x>maxx):
            maxx= x
        if(y<miny):
            miny = y
        if(y>maxy):
            maxy= y
    floor = TEXTURE("parquet1.jpg")(MKPOL([[(minx,miny,0),(maxx,miny,0),(minx,maxy,0),(maxx,maxy,0)],[[1,2,3,4]],1]))
    return floor

#"""lettore di file che restitusce un array di punti non ancora scalati e di tipo float a partire da un nome file passato in input"""
def pointsReader(file):
    temp = []
    text  = open(file).read()
    for line in text.split('\n'):
        splits = line.split()
        #if  len(splits) ==1 and splits[0]== line.strip():
        splits = [float(item) for item in line.strip().split(',') ]
        temp.extend(splits)
    return temp

def doors(array):
    l = len(array)
    wall = CUBOID([0,0,0])
    j = 1
    temp = []
    while(j<l):
        temp.append([j,j+1])
        j+=2
    wall =MKPOL([array,temp,1])
    wall = OFFSET([.1,.1,2])(wall)
    return wall

def windows(array):
    l = len(array)
    wall = CUBOID([0,0,0])
    temp =CUBOID([0,0,0])
    j = 1
    temp = []
    while(j<l):
        temp.append([j,j+1])
        j+=2
    wall =MKPOL([array,temp,1])
    wall = OFFSET([.6,.6,1.5])(wall)
    wall = STRUCT([T(3)(1),wall])
    return wall

In [10]:
#"""vecchia funziona relativa al workshop del tetto"""
def sideBand(points1,points2):
    struct = CUBOID([0,0,0])
    l = len(points1)
    for i in range(0,l-1):
        points =[points1[i],points2[i],points1[i+1],points2[i+1]]
        temp =TEXTURE("canadesi.jpg") (MKPOL([points,[[1,2,3,4]],1]))
        struct = STRUCT([temp,T(0)(0),struct])
    temp = TEXTURE("canadesi.jpg")(MKPOL([[points1[l-1],points2[l-1],points1[0],points2[0]],[[1,2,3,4]],1]))
    struct = STRUCT([struct,T(0)(0),temp])
    return struct

building: funzione che prende in input un array di 4 punti, un numero n che rappresenta il numero di piani, e un hpc che rappresenta l'appartamento. Dapprima crea il soppalco, le scale e il parapetto e successivamente unisce il tutto per poi ripetere il procedimento n volte traslando la struttura creata. Alla fine crea il tetto del palazzo.

In [117]:
def building(points,n,apartament):
    soppalco = COLOR([224/255,167/255,0])(OFFSET([.1,.1,.3])(MKPOL([points,[[1,2,3,4]],1])))
    temp = CUBOID([0,0,0])
    building = CUBOID([0,0,0])
    
    ax,ay,az = points[0]
    bx,by,bz = points[1]
    cx,cy,cz = points[2]
    dx,dy,dz = points[3]

    sdx = 0.8
    sdy = 0.2
    sdz = 0.25
    sn = 16
    stair =  stairs(sdx,sdy,sdz,sn)
    corrimano = [(sdx,0,0),(sdx+0.1,0,0),(sdx,sdy+0.1,0),(sdx+0.1,sdy+0.1,0),(sdx,0,sdz+0.7),(sdx+0.1,0,sdz+0.7),(sdx,sdy+0.1,sdz+0.7),(sdx+0.1,sdy+0.1,sdz+0.7)]
    handrail = MKPOL([corrimano,[[1,2,3,4,5,6,7,8]],1])
    corrimano1 = [(sdx,0,sdz+0.7),(sdx+0.1,0,sdz+0.7),(sdx,sdy+0.1,sdz+0.7),(sdx+0.1,sdy+0.1,sdz+0.7),(sdx,(sdy*sn)+1,3.3),(sdx+0.1,(sdy*sn)+1,3.3),(sdx,(sdy*sn)+0.5,3.3),(sdx+0.1,(sdy*sn)+0.5,3.3)]
    handrail1 = MKPOL([corrimano1,[[1,2,3,4,5,6,7,8]],1])
    handrail= TEXTURE("ringhiera.jpg")(STRUCT([handrail,handrail1]))
        
    stair = STRUCT([R([1,2])(3.14),stair])

    apartament =STRUCT([temp,T([1,2,3])([0,(-(ay+cy)/4),.301]),apartament]) 
    
    soppalco = STRUCT([temp,T(2)(-(ay+cy)/4),soppalco])
    soppalco = STRUCT([apartament,soppalco])
    
    diffPlatform = CUBOID([1.2,+2.5,.3])
    diffPlatform = STRUCT([temp,T([1,2,3])([0.3,(0.2*15)+1,0]),diffPlatform])
    platform = OFFSET([.1,.1,.3])(MKPOL([[(ax,ay,0),(0,ay,0),(0,cy,0),(cx,cy,0)],[[1,2,3,4]],1]))
    platform = STRUCT([temp,T([1,2,3])([-0.1,(-(ay+cy)/4)]),platform])  
    platform = TEXTURE("pianerottolo.jpg")(DIFFERENCE([platform,diffPlatform]))
    
    building = STRUCT([platform,soppalco])
    
    parapet = TEXTURE("muretto.jpg")(OFFSET([0,0,1])(MKPOL([[(0,cy,0.3),(0.2,cy,0.3),(0,ay,0.3),(0.2,ay,0.3)],[[1,2,3,4]],1])))
    parapet = STRUCT([temp,T(2)((-(ay+cy)/4)),parapet])
    parapet1 = TEXTURE("muretto.jpg")(OFFSET([0,0,1])(MKPOL([[(0,cy,0.3),(0,cy-0.2,0.3),(cx,cy,0.3),(cx,cy-0.2,0.3)],[[1,2,3,4]],1])))
    parapet2 = TEXTURE("muretto.jpg")(OFFSET([0,0,1])(MKPOL([[(0,ay,0.3),(0,ay+0.2,0.3),(ax,ay,0.3),(ax,ay+0.2,0.3)],[[1,2,3,4]],1])))
    parapet = STRUCT([parapet,T(2)((-(ay+cy)/4)),parapet1])
    parapet = STRUCT([parapet,T(2)((-(ay+cy)/4)),parapet2])
    building = STRUCT([building,parapet])
    building = STRUCT([building,T([1,2,3])([cx-0.5,ay+1.8,-3]),stair])
    if (n>2):
        for i in range(0,n-2):
            building = STRUCT([building,T(3)(3.3),building])
    roof = MKPOL([[(0,ay,az),points[1],points[3],(0,cy,cz)],[[1,2,4,3]],1])
    d=1
    h=1
    roofPoints = [(0,ay,az),points[1],points[3],(0,cy,cz)]
    roofPoints1 = [(0+d,ay+d,h),(bx-d,by+d,h),(dx-d,dy-d,h),(0+d,cy-d,h)]
    side = sideBand(roofPoints,roofPoints1)
    roof1 = MKPOL([roofPoints1,[[1,2,3,4]],1])    
    roof1 = TEXTURE("terrazzo1.jpg")(roof1)
    finalStructure = STRUCT([side,T(0)(0),roof1])
    finalRoof = STRUCT([finalStructure,T(0)(0),roof])
    finalRoof = STRUCT([temp,T([1,2,3])([0,(-(ay+cy)/4),.301]),finalRoof])
    building = STRUCT([building,T(3)((3.3*(n-1)-0.3)),finalRoof])
    return building


firstFloor(points,n,apartament): prende in input un array di 4 punti, un numero n e un hpc che rappresenta l'appartamento.
crea il garage e unisce il primo piano con il garage e le colonne del primo piano.

In [118]:

#"""prende in input un array di 4 punti e crea il garage e il primo piano con le scale"""
def firstFloor(points,n,apartament):
    gar = COLOR([224/255,167/255,0])(OFFSET([.1,.1,3])(MKPOL([points,[[1,2,3,4]],1])))
    temp = CUBOID([0,0,0])
    stair =  stairs(0.8,0.2,0.25,15)
    ax,ay,az = points[0]
    bx,by,bz = points[1]
    cx,cy,cz = points[2]
    dx,dy,dz = points[3]
    column = OFFSET([1,1,3])(MKPOL([[(0,cy,0),(0.5,cy,0),(0,cy-0.5,0),(0.5,cy,0)],[[1,2,3,4]],1]))
    column =TEXTURE("pianerottolo.jpg")(STRUCT([temp,T(1)(0),column]))
    column1 = TEXTURE("pianerottolo.jpg")(CUBOID([.29,.8,2.5]))
    column1 = STRUCT([T(2)(1.3),column1])
    
    stair = STRUCT([temp,T(1)(0.3),stair])
    middlePoint = (ax+bx)/2
    garageDoor = TEXTURE("garage.jpg")(OFFSET([.1,.1,2.2])(MKPOL([[(ax+2,ay,az),(middlePoint-1,ay,az)],[[1,2]],1])))
    garageDoor1 = TEXTURE("garage.jpg")(OFFSET([.1,.1,2.2])(MKPOL([[(middlePoint+1,ay,az),(bx-2,ay,az)],[[1,2]],1])))
    gar = STRUCT([gar,garageDoor,garageDoor1])
    
    diffPlatform = CUBOID([1.2,-2.5,.5])
    diffPlatform = STRUCT([temp,T([1,2,3])([0.3,(0.2*15)+0.6,3-0.5]),diffPlatform])
    platform = OFFSET([.1,.1,.5])(MKPOL([[(ax,ay,3),(0,ay,3),(0,cy,3),(cx,cy,3)],[[1,2,3,4]],1]))
    platform = STRUCT([temp,T([1,2,3])([-0.1,(-(ay+cy)/4),-0.5]),platform])   
    platform = TEXTURE("pianerottolo.jpg")(DIFFERENCE([platform,diffPlatform]))
    platform = STRUCT([stair,platform])
    
    platform = STRUCT([platform,T(2)(-(ay+cy)/4),gar])
    platform = STRUCT([platform,T(2)(-(ay+cy)/4-1),column])
    platform = STRUCT([platform,column1])
    
    parapet = TEXTURE("muretto.jpg")(OFFSET([0,0,1])(MKPOL([[(0,cy,3),(0.2,cy,3),(0,ay+(0.2*15),3),(0.2,ay+(0.2*15),3)],[[1,2,3,4]],1])))
    parapet1 = TEXTURE("muretto.jpg")(OFFSET([0,0,1])(MKPOL([[(0,cy,3),(0,cy-0.2,3),(cx,cy,3),(cx,cy-0.2,3)],[[1,2,3,4]],1])))
    parapet = STRUCT([parapet1,T(2)(0),parapet])
    
    platform = STRUCT([platform,T([2,3])([(-(ay+cy)/4),0]),parapet])
    platform = STRUCT([platform,T([1,2,3])([0,(-(ay+cy)/2)/2,3]),apartament])
    build = CUBOID([0,0,0])
    build = building(points,n,apartament)
    platform = STRUCT([platform,T(3)(6),build])
    return platform



ggpl_workshop_10(fileHome,fileDoor,fileWindow,n): Funzione principale che prende in input 3 file .txt di linee, rispettivamente per la casa, le porte e le finestre e un intero n che rappresenta il numero di piani.
All'interno del metodo richiama tutte le funzioni di supporto sopra descritte ma principalmente inserisce nell'appartamento le porte e le finestre.
Come condizione principale si richiede che l'appartamento sia un rettangolo.

In [122]:
def ggpl_workshop_10(fileHome,fileDoor,fileWindow,n):
    temp = CUBOID([0,0,0])
    homePoints = pointsReader(fileHome)
    homePoints = makePoints(homePoints)
    maxMin =findMaxMin(homePoints)
    ax,ay,az = maxMin[0]
    bx,by,bz = maxMin[1]
    cx,cy,cz = maxMin[2]
    dx,dy,dz = maxMin[3]
    wall = walls(homePoints)
    
    doorPoints = pointsReader(fileDoor)
    doorPoints = makePoints(doorPoints)
    door = doors(doorPoints)
   
    
    windowsPoints = pointsReader(fileWindow)
    windowsPoints = makePoints(windowsPoints)
    window = windows(windowsPoints)
    structure = COLOR([224/255,167/255,0])(DIFFERENCE([wall,door]))
    structure = COLOR([224/255,167/255,0])(DIFFERENCE([structure,window]))
    f = floor(homePoints)
    home = (STRUCT([f,structure]))
    #"""porta usata """
    X=[.1,.1,.1]
    Y=[.35,.6,.35]
    Z=[.25,.35,.25,.35,.25,.35,.25]
    occ1 = [[1,1,1],[1,2,1],[1,1,1],[1,2,1],[1,1,1],[1,2,1],[1,1,1]]
    occ = [occ1]
    porta = ggpl(X,Y,Z,occ)(.09,.9,.9)
    j = 0
    tempDoor=[]
    l =len(doorPoints)
    while j<l-1:
        tempDoor.append([doorPoints[j],doorPoints[j+1]])
        j=j+2
    portaR = STRUCT([R([1,2])(-(3.14/2)),porta])
    d =tempDoor[0]
    a,b = d
    x,y,z= a
    home =STRUCT([home,T([1,2])([x,y]),porta])
    j=1
    #"""inserisco le porte"""
    while j<=(len(tempDoor)-1):
        d =tempDoor[j]
        a,b = d
        x,y,z= a
        x1,y1,z1= b
        home =STRUCT([home,T([1,2])([x,y]),portaR])
        j = j+1
    #"""finestre usate"""
    occ1 = [[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1]]
    occ2 = [[1,1,1,1,1],[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[1,1,1,1,1]]
    occ3 = [[1,1,1,1,1],[0,2,1,2,0],[0,2,1,2,0],[0,2,1,2,0],[1,1,1,1,1]]
    occ4 = [[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1]]
    occ = [occ1, occ2,occ3, occ4]
    finestra = ggpl([.1,.015,.015,.1],[.4,.5,.05,.5,.4],[.15,.6,.05,.6,.15],occ)(.09,.9,.9)
    finestra = STRUCT([R([1,2])(-(3.14/2)),finestra])
    j = 0
    tempWindows=[]
    l =len(windowsPoints)
    while j<l:
        tempWindows.append([windowsPoints[j],windowsPoints[j+1]])
        j=j+2
    d = tempWindows[0]
    a,b = d
    x,y,z= a
    j=0
    #"""inserisco le finestre"""
    while j<=(len(tempWindows)-1):
        d =tempWindows[j]
        a,b = d
        x,y,z= a
        x1,y1,z1= b
        home =STRUCT([home,T([1,2,3])([x,y,1]),finestra])
        j = j+1
    home = firstFloor(maxMin,n,home)
    return home


VIEW(ggpl_workshop_10("lineporta.txt","lineport.txt","linefinestra.txt",6))


<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000009FD2600> >

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000009FCA0C0> >